# Stationary Droplet and Translating Droplet
## Test case descriptions
For a description of the test cases and their setup, please refer to the preprint/paper.

## Evaluation
Since the stationary droplet and the translating droplet, both in 2D and 3D, employ the same error metrics
their evaluation is summerized in one notebook.  
Mandatory and optional data columns for the CSV files providing the plot data. See the repository `README.md`
for general syntax information.

| column name | description | required |
| :-          | :-          | :-       |
| `solver` | solver name, e.g. *interFoam* | yes |
| `fluid_pairing` | name the two phases used for simulation, e.g. water-air | yes |
| `resolution` | mesh resolution as number of cells per spatial direction | yes | 
| `time` | physical time of the data point in seconds | yes |
| `mean_absolute_error_velocity` | See paper | yes |
| `root_mean_square_deviation_velocity` | See paper | no |
| `max_error_velocity` | See paper | yes |


## Usage
The notebook is prepared such that you can just hit `Kernel` ->
`Restart & Run All`. This plots the temporal evolution of the maximum velocity error and mean velocity error for all fluid/fluid pairings, resolutions and solvers, as well as the more distribution plots (violin plots).  

### Customize output
Add each test case for which you want to create plots, e.g. _stationaryDroplet2D_ and _translatingDroplet3D_,
to the `test_cases` list.  
Add each error metric for which you want to create plots, e.g. *max_velocity_error* and *mean_absolute_velocity_error*
the the `error_metrics` list.  
For each combination (test case, error metric), plots for the different resolutions and fluid pairings are created, as well as the error distribution in form of violin plots.  
Per default, all solvers for which data is available are added to each plot. If you want to compare a specific set of solvers, remove the `'all'` entry from `solvers` below and list the solvers you want to compare, e.g.
`solvers = ['interFoam', 'Fluent']`. If you prescribe a solver subset and a solver is missing in the plots, make sure that its spelling matches the entries in the CSV file exactly.

In [ ]:
# NOTE:
# Activating all cases and error metrics at once may result in severe
# memory consumption.
# You have been warned.
test_cases = ['stationaryDroplet2D',
              'stationaryDroplet3D',
              'translatingDroplet2D',
              'translatingDroplet3D'
             ]

# Path to directory containing test case subfolders with CSV data
directory = 'data/hydrodynamic/'

# Metadata columns
index_columns = ['solver', 'fluid_pairing', 'resolution']

# Data columns
data_columns = ['time',
                'mean_absolute_error_velocity',
                #'root_mean_square_deviation_velocity',
                'max_error_velocity'
               ]

# Solver selection
solvers = ['all']

# Error metrics for which plots are created
error_metrics = list(data_columns[1:])

In [ ]:
from shared_functions import *
from itertools import product
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import subprocess

plt.rcParams.update(latex_settings)
plt.rcParams.update({'figure.max_open_warning': 0})

# Each parameter combination is used to create a separate figure
group_by_parameters = ["fluid_pairing", "resolution"]

def create_plots(df, test_case, error_metric, group_by_parameters):
    # Marker and line styles
    n_marks = 10
    solvers = list(df.index.unique(level='solver'))
    solver_marker, solver_line = assign_styles_to_solvers(solvers)

    for params, subsetdf in  df.groupby(level=group_by_parameters):
        fig, ax = plt.subplots(figsize=figureSize, dpi=DPI)
    
        for solver, plotdf in subsetdf.groupby(level="solver"):
            ax.semilogy(plotdf["time"], plotdf[error_metric],
                    label=solver,
                    marker=solver_marker[solver], markevery=int(len(plotdf.index)/n_marks),
                    linestyle=solver_line[solver])    
        
        # Set axis labels
        ax.set_ylabel(str(error_metric).replace('_', ' ') + " [m/s]", fontsize=plotfontsize)
        ax.set_xlabel("time [s]", fontsize=plotfontsize)
        ax.tick_params(axis='both', labelsize=plotfontsize)
        
        ax.grid(which="major")
        ax.legend(fontsize=plotfontsize)
        
        title = title_from_parameters(group_by_parameters, params, test_case)

        if not os.path.isdir(figure_directory):
            os.mkdir(figure_directory)
        fig.savefig(os.path.join(figure_directory, format_for_filename(title) + 
                                 "_" + error_metric + ".pdf"), bbox_inches="tight")
        ax.set_title(title, fontsize=plotfontsize)

        
def create_table(df, test_case, error_metric, index_level_names):
    table_columns = list(index_level_names) + \
                    [r"mean",
                     r"$\epsilon^t_\text{min}\left(\v\right)$",
                     r"$\epsilon^t_\text{max}\left(\v\right)$",
                     r"tendency"]
    table_df = pd.DataFrame(columns=table_columns)
    
    # Only consider the second half of simulated time
    max_time = max(df["time"])
    half_df = df[df["time"] > 0.5*max_time] 
    
    for params, subset_df in half_df.groupby(level=index_level_names):
        mean_velocity = subset_df[error_metric].mean()
        min_velocity = min(subset_df[error_metric])
        max_velocity = max(subset_df[error_metric])
        linear_fit = np.polynomial.Polynomial.fit(subset_df["time"], subset_df[error_metric], 1)
        table_df.loc[len(table_df.index)] = list(params) + [mean_velocity,
                                                            min_velocity,
                                                            max_velocity,
                                                            linear_fit.coef[-1]]
    table_df.set_index(index_columns, inplace=True)
    table_df.sort_index(inplace=True)
    
    # Rename index columns to reduce horizontal table width
    table_df.index.rename("res", level="resolution", inplace=True)
    
    table_directory = "tables"
    if not os.path.isdir(table_directory):
        os.mkdir(table_directory)
    table_file_name = os.path.join(table_directory, test_case+".tex")
    # Format data columns to exponential format with two decimal places
    table_df.style.format('{:.2e}').to_latex(table_file_name, hrules=True, clines="skip-last;data")
    # Remove underscores in non-math text to avoid weird rendering in LaTeX.
    # If the table export settings are changed above, the start line for search and replace
    # (currently 4) has probably to be changed
    subprocess.run(r"sed -i '4,$s/_/ /g' " + str(table_file_name), shell=True)

        
for test_case in test_cases:
    ddir = directory + test_case + "/"
    data_files = [filename for filename in os.listdir(ddir) if filename.endswith(".csv")]
    
    df = read_solver_data(data_files, index_columns, data_columns, data_dir=ddir,
                          solver_subset=solvers)
    
    for error_metric in error_metrics:
        create_plots(df, test_case, error_metric, group_by_parameters)
        # Tables disbaled for now: not used in the paper, only works on Linux currently.
        #create_table(df, test_case, error_metric, index_columns)
        
    # Filter by time: only consider second half of simulated time for violin plots
    max_time = max(df["time"])
    filtered_df = df[df["time"] > 0.5*max_time]
    create_violin_plots(filtered_df, test_case, error_metrics, yscale='log',
                        ylabel='velocity error [m/s]', file_name_addendum='_mean_and_max_error')